In [4]:
#관전포인트 : 경험모을 때 gpu 를 사용할 수 있는가

In [1]:
import math
import random

import gym
import numpy as np

import torch
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F
from torch.distributions import Normal

np.random.permutation(6)

np.split()

ids = np.split(ids[:batch_size // mini_batch_size * mini_batch_size], batch_size // mini_batch_size)

In [2]:
from IPython.display import clear_output
import matplotlib.pyplot as plt
%matplotlib inline

<h2>Use CUDA</h2>

In [3]:
use_cuda = torch.cuda.is_available()
device   = torch.device("cuda:1" if use_cuda else "cpu")

<h2>Create Environments</h2>

In [4]:
from common.multiprocessing_env import SubprocVecEnv

num_envs = 16
env_name = "Pendulum-v0"

def make_env():
    def _thunk():
        env = gym.make(env_name)
        return env

    return _thunk

envs = [make_env() for i in range(num_envs)]
envs = SubprocVecEnv(envs)

env = gym.make(env_name)

<h2>Neural Network</h2>

In [5]:
def init_weights(m):
    if isinstance(m, nn.Linear):
        nn.init.normal_(m.weight, mean=0., std=0.1)
        nn.init.constant_(m.bias, 0.1)
        

class ActorCritic(nn.Module):
    def __init__(self, num_inputs, num_outputs, hidden_size, std=0.0):
        super(ActorCritic, self).__init__()
        
        self.critic = nn.Sequential(
            nn.Linear(num_inputs, hidden_size),
            nn.ReLU(),
            nn.Linear(hidden_size, 1)
        )
        
        self.actor = nn.Sequential(
            nn.Linear(num_inputs, hidden_size),
            nn.ReLU(),
            nn.Linear(hidden_size, num_outputs),
        )
        self.log_std = nn.Parameter(torch.ones(1, num_outputs) * std)
        
#         self.apply(init_weights)
        
    def forward(self, x):
        value = self.critic(x)
        mu    = self.actor(x)
        std   = self.log_std.exp().expand_as(mu)
        dist  = Normal(mu, std)
        return dist, value

In [6]:
def plot(frame_idx, rewards):
    clear_output(True)
    plt.figure(figsize=(20,5))
    plt.subplot(131)
    plt.title('frame %s. reward: %s' % (frame_idx, rewards[-1]))
    plt.plot(rewards)
    plt.show()
    
def test_env(vis=False):
    state = env.reset()
    if vis: env.render()
    done = False
    total_reward = 0
    while not done:
        state = torch.FloatTensor(state).unsqueeze(0).to(device)
        dist, _ = model(state)
        next_state, reward, done, _ = env.step(dist.sample().cpu().numpy()[0])
        state = next_state
        if vis: env.render()
        total_reward += reward
    return total_reward

<h2>GAE</h2>

In [7]:
def compute_gae(next_value, rewards, masks, values, gamma=0.99, tau=0.95):
    values = values + [next_value]
    gae = 0
    returns = []
    for step in reversed(range(len(rewards))):
        delta = rewards[step] + gamma * values[step + 1] * masks[step] - values[step]
        gae = delta + gamma * tau * masks[step] * gae
        returns.insert(0, gae + values[step])
    return returns

<h1> Proximal Policy Optimization Algorithm</h1>
<h2><a href="https://arxiv.org/abs/1707.06347">Arxiv</a></h2>

In [8]:
def ppo_iter(mini_batch_size, states, actions, log_probs, returns, advantage): # 전체 데이터 셋에서 mini_batch 를 만드는 것이다.
    batch_size = states.size(0)
    ids = np.random.permutation(batch_size)
    ids = np.split(ids[:batch_size // mini_batch_size * mini_batch_size], batch_size // mini_batch_size)
    for i in range(len(ids)):
        yield states[ids[i], :], actions[ids[i], :], log_probs[ids[i], :], returns[ids[i], :], advantage[ids[i], :]

def ppo_update(ppo_epochs, mini_batch_size, states, actions, log_probs, returns, advantages, clip_param=0.2): # training
    for _ in range(ppo_epochs):
        for state, action, old_log_probs, return_, advantage in ppo_iter(mini_batch_size, states, actions, log_probs, returns, advantages):
            dist, value = model(state)
            entropy = dist.entropy().mean()
            new_log_probs = dist.log_prob(action)

            ratio = (new_log_probs - old_log_probs).exp()
            surr1 = ratio * advantage
            surr2 = torch.clamp(ratio, 1.0 - clip_param, 1.0 + clip_param) * advantage

            actor_loss  = - torch.min(surr1, surr2).mean()
            critic_loss = (return_ - value).pow(2).mean()

            loss = 0.5 * critic_loss + actor_loss - 0.001 * entropy

            optimizer.zero_grad()
            loss.backward()
            optimizer.step()

In [9]:
num_inputs  = envs.observation_space.shape[0]
num_outputs = envs.action_space.shape[0]

#Hyper params:
hidden_size      = 32
lr               = 1e-3
num_steps        = 128
mini_batch_size  = 256
ppo_epochs       = 30
threshold_reward = -100

model = ActorCritic(num_inputs, num_outputs, hidden_size).to(device)
optimizer = optim.Adam(model.parameters(), lr=lr)

In [10]:
max_frames = 15000
frame_idx  = 0
test_rewards = []
early_stop = False

In [11]:
state = envs.reset()

In [15]:


while frame_idx < 20000 and not early_stop: #15000 epoch 반복

    log_probs = []
    values    = []
    states    = []
    actions   = []
    rewards   = []
    masks     = []
    entropy = 0

    for _ in range(num_steps): #경험 모으기 - gpu 쓰는구나 . 하지만 여전히 DataParallel 들어갈 여지는 없어보인다. 
        #-> 아.. state 가 벡터 1개가 아닐 것 같다.. 16개네. gpu 쓸만하다. DataParallel 도 가능할듯?
        state = torch.FloatTensor(state).to(device)
        dist, value = model(state)

        action = dist.sample()
        next_state, reward, done, _ = envs.step(action.cpu().numpy()) #done 한다고 끝내질 않네??
        

        log_prob = dist.log_prob(action)
        entropy += dist.entropy().mean()
        
        log_probs.append(log_prob)
        values.append(value)
        rewards.append(torch.FloatTensor(reward).unsqueeze(1).to(device))
        masks.append(torch.FloatTensor(1 - done).unsqueeze(1).to(device))
        
        states.append(state)
        actions.append(action)
        
        state = next_state
        frame_idx += 1
        
        if frame_idx % 1000 == 0: # 1000번 마다 plot 그려주기
            test_reward = np.mean([test_env() for _ in range(10)])
            test_rewards.append(test_reward)
            torch.save(model.state_dict(),"weight/updated_ppo_weight_{}.pth".format(frame_idx))
            plot(frame_idx, test_rewards)
            if test_reward > threshold_reward: early_stop = True

    print("states", len(states))
    print("state_shape" , state.shape)
    print("state" , state)
    print("next_state:", next_state)
    break
    #경험 1세트 모은거로 학습하기
    
    #num_step 만큼 진행했을 때 reward 얼마였는지 구하기
    next_state = torch.FloatTensor(next_state).to(device) 
    _, next_value = model(next_state)
    returns = compute_gae(next_value, rewards, masks, values)
    

    returns   = torch.cat(returns).detach()
    log_probs = torch.cat(log_probs).detach()
    values    = torch.cat(values).detach()
    states    = torch.cat(states)
    actions   = torch.cat(actions)
    advantage = returns - values
    
    ppo_update(ppo_epochs, mini_batch_size, states, actions, log_probs, returns, advantage)

states 128
state_shape (16, 3)
state [[ 0.84686884 -0.53180181 -3.72222422]
 [-0.5837327  -0.81194589 -5.51436465]
 [ 0.97199038  0.23502065 -4.74187172]
 [ 0.9878556  -0.15537472 -4.8302156 ]
 [ 0.39222805 -0.91986801 -3.81595192]
 [ 0.07569987  0.99713065 -6.67020438]
 [ 0.59709577 -0.80216996 -5.37162599]
 [-0.49253922 -0.87029025 -7.59325613]
 [-0.87450187  0.48502214  0.90836877]
 [ 0.85135621 -0.52458803 -4.65209622]
 [ 0.85719893  0.51498544 -4.80911915]
 [-0.25246806  0.96760523 -3.58420635]
 [-0.68369031 -0.72977227 -1.50603343]
 [-0.09939207  0.99504835 -7.04099248]
 [ 0.09976505 -0.99501102  5.42113061]
 [-0.50714327  0.86186177 -7.72818957]]
next_state: [[ 0.84686884 -0.53180181 -3.72222422]
 [-0.5837327  -0.81194589 -5.51436465]
 [ 0.97199038  0.23502065 -4.74187172]
 [ 0.9878556  -0.15537472 -4.8302156 ]
 [ 0.39222805 -0.91986801 -3.81595192]
 [ 0.07569987  0.99713065 -6.67020438]
 [ 0.59709577 -0.80216996 -5.37162599]
 [-0.49253922 -0.87029025 -7.59325613]
 [-0.87450187 

<h1>Saving trajectories for GAIL</h1>

In [17]:
from itertools import count

max_expert_num = 50000
num_steps = 0
expert_traj = []

for i_episode in count():
    state = env.reset()
    done = False
    total_reward = 0
    
    while not done:
        state = torch.FloatTensor(state).unsqueeze(0).to(device)
        dist, _ = model(state)
        action = dist.sample().cpu().numpy()[0]
        next_state, reward, done, _ = env.step(action)
        state = next_state
        total_reward += reward
        expert_traj.append(np.hstack([state, action]))
        num_steps += 1
    
    print("episode:", i_episode, "reward:", total_reward)
    
    if num_steps >= max_expert_num:
        break
        
expert_traj = np.stack(expert_traj)
print()
print(expert_traj.shape)
print()
np.save("expert_traj.npy", expert_traj)

episode: 0 reward: -6.005084751638878
episode: 1 reward: -126.8927871226066
episode: 2 reward: -133.36746402003962
episode: 3 reward: -266.05918127231354
episode: 4 reward: -339.21731628644164
episode: 5 reward: -131.8278783064484
episode: 6 reward: -248.87367431578727
episode: 7 reward: -137.175333194117
episode: 8 reward: -6.328519695053957
episode: 9 reward: -137.1696418909525
episode: 10 reward: -252.9144404954698
episode: 11 reward: -129.00562978825454
episode: 12 reward: -268.5682250542064
episode: 13 reward: -395.79137086108
episode: 14 reward: -254.1389557076782
episode: 15 reward: -135.80559049099568
episode: 16 reward: -136.58210836012515
episode: 17 reward: -256.44810599440734
episode: 18 reward: -260.0060756338123
episode: 19 reward: -133.67263772828153
episode: 20 reward: -129.79663430485184
episode: 21 reward: -130.3333707007226
episode: 22 reward: -265.65811345573445
episode: 23 reward: -135.6062894944192
episode: 24 reward: -264.2079099342339
episode: 25 reward: -131.09

episode: 207 reward: -125.15593957762067
episode: 208 reward: -253.0425249314722
episode: 209 reward: -268.8299869541173
episode: 210 reward: -252.8473403617894
episode: 211 reward: -132.61120193546267
episode: 212 reward: -270.9579964352161
episode: 213 reward: -131.34855669213113
episode: 214 reward: -131.00768935733026
episode: 215 reward: -132.92985278472182
episode: 216 reward: -132.84019123412398
episode: 217 reward: -124.8756838298895
episode: 218 reward: -129.8676487092138
episode: 219 reward: -133.33154871056428
episode: 220 reward: -130.67092641961224
episode: 221 reward: -298.8569392495992
episode: 222 reward: -445.42861013695665
episode: 223 reward: -259.60402354894927
episode: 224 reward: -6.99545469709659
episode: 225 reward: -133.2262897376931
episode: 226 reward: -256.996547530838
episode: 227 reward: -129.4792150247529
episode: 228 reward: -642.5008652630114
episode: 229 reward: -133.4147409761136
episode: 230 reward: -418.2579560907369
episode: 231 reward: -368.958995

In [48]:
pi[action]

tensor([[-0.1179, -0.1179,  0.1210, -0.2299,  0.1210, -0.1179,  0.5470,  0.5470,
          0.1210,  0.5154, -0.2299,  0.1210,  0.5154,  0.5470, -0.2299, -0.1179],
        [ 0.3900,  0.3900, -0.8305,  0.2822, -0.8305,  0.3900,  1.8698,  1.8698,
         -0.8305, -1.0892,  0.2822, -0.8305, -1.0892,  1.8698,  0.2822,  0.3900],
        [-0.2651, -0.2651,  0.2852, -0.5086,  0.2852, -0.2651,  0.1309,  0.1309,
          0.2852, -0.5207, -0.5086,  0.2852, -0.5207,  0.1309, -0.5086, -0.2651],
        [-0.1693, -0.1693, -1.5956,  1.3470, -1.5956, -0.1693, -0.3691, -0.3691,
         -1.5956, -0.9737,  1.3470, -1.5956, -0.9737, -0.3691,  1.3470, -0.1693],
        [ 0.5413,  0.5413,  0.2120,  0.5419,  0.2120,  0.5413, -0.8155, -0.8155,
          0.2120, -1.7334,  0.5419,  0.2120, -1.7334, -0.8155,  0.5419,  0.5413],
        [ 1.1363,  1.1363, -2.1591, -0.2004, -2.1591,  1.1363,  1.1208,  1.1208,
         -2.1591,  0.4594, -0.2004, -2.1591,  0.4594,  1.1208, -0.2004,  1.1363],
        [-0.7458, -0.7